**LICENCE:**
This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.

**SYNTAX:**
You can enter `→` by writing `->` and pressing TAB; pressing TAB again goes back to `->`. Similar useful combinations are:

| base form | alternate form |
|:---------:|:--------------:|
| ->  | → |
| \   | λ |
| neg | ¬ |
| top | ⊤ |
| bot | ⊥ |
| /\  | ∧ |
| \\/ | ∨ |
| <   | ⟨ |
| >   | ⟩ |
| forall | ∀ |
| exists | ∃ |
| Pi  | Π |
| Sigma | Σ |

# Curry-Howard isomorphism

| Logic    | Programming |     |
|----------|-------------|-----|
| theorems | types       | $A$ |
| proofs   | programs (terms) | $a$ |
| proof simplification | program execution | run $a$ |
| proof development | programming | write $a$ |
| proof checking | type checking | given $a, A$, does $a : A$ hold? |
| validity | type inhabitation | given $A$, find $a$ s.t. $a : A$ |
| ?        | type reconstruction | given $a$, find $A$ s.t. $a : A$ |

# Intuitionistic propositional logic

We reproduce without further comment the basic definitions for propositional logic.

````
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell1 where

infixr 2 _∧_
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B

fst : {A B : Set} → A ∧ B → A
fst ⟨ a , _ ⟩ = a

snd : {A B : Set} → A ∧ B → B
snd ⟨ _ , b ⟩ = b

infixr 1 _∨_ 
data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B

case : {A B C : Set} → (A → C) → (B → C) → A ∨ B → C
case f g (left x) = f x
case f g (right x) = g x

data ⊤ : Set where
  tt : ⊤

A→⊤ : {A : Set} → A → ⊤
A→⊤ _ = tt

data ⊥ : Set where

⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

infix 3 ¬_ 
¬_ : Set → Set
¬ A = A → ⊥
````

# Dependent types in Agda

So far we have only seen *(polymorphic) simple types*,
such as `A → B` for `(A B : Set)`.
There are situations where one wants to impose some additional property on the output `B`
which depends on the particular input `a : A`.
In order to achieve this, the output `B` is generalised to have type `A → Set` (instead of just `Set`).
One obtains *dependent* function space

```agda
(a : A) → B a
```

Notice how the type expression `B a` now contain the *term* `a`,
i.e., the type `B a : Set` *depends* on `a`.

Our main application and source of examples of dependent types is to model intuitionistic first-order logic.

# Intuitionistic first-order logic

We show how dependent types can be used to implement universal quantification (dependent function space)
and existential quantification (dependent product).

## The universal quantifier ∀

In intuitionistic logic a proof of $\forall (a : A) B$ is a function $f$ mapping a proof $a$ of $A$
into a proof $f\; a$ of $B\; a$, where we can see $B$ as a family of types indexed by proofs of $A$.
The universal quantifier is implemented via the *dependent function space*

````
Π : (A : Set) → (B : A → Set) → Set
Π A B = (a : A) → B a
````

```agda
-- compare with implication:
((a : A) → B a) versus A → B
```

This generalises implication `A → B`, which corresponds to non-dependent function space.
In this sense, in intuitionistic logic implication is a special case of universal quantification.

(Universal quantification also generalises conjunction,
since the type $B_1 \wedge B_2$ is isomorphic to $\Pi\; A\; B$
where $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, sometimes `Π` is called dependent product, hence the notation.
However, for us dependent product is something else and it will be used below to model existential quantification.)

````
-- the type of the first argument of Π can be inferred from the second
forAll : {A : Set} → (B : A → Set) → Set
forAll {A} B = Π A B

-- we introduce a convenient syntax reminiscent of universal quantification in logic
∀-syntax = forAll
infix 0 ∀-syntax
syntax ∀-syntax (λ x → B) = ∀[ x ] B

-- dependent apply; corresponds to ∀-elimination
apply : {A : Set} → {B : A → Set} → Π A B → (a : A) → B a
apply f x = f x
````

### **Exercise** (`∀`)

1. Show that the universal quantifier commutes with itself:
$(\forall a. \forall b. P\; a\; b) \to \forall b. \forall a. P\; a\; b$.

2. Show that one can diagonalise a universal relation:
$(\forall a. \forall b. P\; a\; b) \to \forall a. P\; a\; a$.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell2 where
open import code.cell1 public

-- recall that "∀[ a ] ∀[ b ] P a b" is the same as
-- "Π A (λ a → Π B (λ b → P a b))", which in turn just means
-- "(a : A) → (b : A) → P a b"
∀-comm : {A B : Set} {P : A → B → Set} → ∀[ a ] ∀[ b ] P a b → ∀[ b ] ∀[ a ] P a b
∀-comm = ?

∀-diag : {A : Set} {P : A → A → Set} → ∀[ a ] ∀[ b ] P a b → ∀[ a ] P a a
∀-diag = ?

### **Exercise** (`∀` and `→`)

Show that the universal quantifier distributes over implication:
$(\forall a. P\; a\ \to Q\; a) \to (\forall a. P\; a) \to \forall a. Q\; a$.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell3 where
open import code.cell2 public

-- recall that "∀[ a ] P a" is the same as
-- "Π A (λ a → P a)", which in turn just means
-- "(a : A) → P a"
∀→-distr : {A : Set} {P Q : A → Set} → ∀[ a ] (P a → Q a) → ∀[ a ] P a → ∀[ a ] Q a
∀→-distr = ?

### **Exercise** (`∀` and  `∧`)

Prove the following intuitionistic tautologies.

1. $\forall a. B \wedge C \to (\forall a. B) \wedge (\forall a. C)$.
2. $(\forall a. B) \wedge (\forall a. C) \to \forall a. B \wedge C$.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell4 where
open import code.cell3 public

-- 1.
∀∧-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∧ C a → (∀[ a ] B a) ∧ (∀[ a ] C a)
∀∧-distr = ?

{-
The notation "∀[ a ] B a" is just syntactic sugaring for "Π A (λ a → B a)"
(where A is inferred automatically from the type of B),
which in turn is defined to be "(a : A) → (λ a → B a) a",
i.e., "(a : A) → B a".

Using "∀[ a ] B a" instead of "(a : A) → B a" makes the syntax closer to logic.
-}

-- 2.
∧∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∧ (∀[ a ] C a) → ∀[ a ] B a ∧ C a
∧∀-distr = ?

### **Exercise** (`∀` and `∨`)

Which of the following are intuitionistic tautologies?
Which are classic tautologies?
Which are neither?
Prove the intuitionistic ones.

1. $\forall a B \vee C \to (\forall a B) \vee (\forall a C)$.
2. $\forall a B \vee C \to (\forall a B) \vee C$, where $a$ does not occur in $C$.
3. $(\forall a B) \vee (\forall a C) \to \forall a (B \vee C)$.

*Hint:* If you cannot easily program a solution, then most likely there is no solution.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell5 where
open import code.cell4 public

-- 1.
∀∨-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∨ C a → (∀[ a ] B a) ∨ (∀[ a ] C a)
∀∨-distr = ?

-- 2.
∀∨-distr' : {A C : Set} {B : A → Set} → ∀[ a ] B a ∨ C → (∀[ a ] B a) ∨ C
∀∨-distr' = cheat where postulate cheat : _

-- 3.
∨∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∨ (∀[ a ] C a) → ∀[ a ] B a ∨ C a
∨∀-distr = ?

### **Exercise** (`∀` and `¬`)

The *double negation shift* (DNS) is the following classical tautology:
$(\forall a. \neg\neg P\; a) \to \neg\neg \forall a. P\; a$.
While DNS does not hold intuitionistically, prove that its converse does.


In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell6 where
open import code.cell5 public

-- converse double negation shift
cdns : {A : Set} {P : A -> Set} → ¬ ¬ (∀[ a ] P a) → ∀[ a ] ¬ ¬ P a
cdns = ?

## The existential quantifier ∃

In intuitionistic logic,
a proof of $\exists (a : A) B$ is a pair $(a, b)$,
where $a$ is a proof of $A$
and $b$ is a proof of $B\; a$.
Like in universal quantification, we can see $B$ as a family of types indexed by proofs of $A$.
The existential quantifier is implemented with the *dependent product*:

````
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell7 where
open import code.cell6 public

data Σ (A : Set) (B : A → Set) : Set where
    ⟨_,_⟩ : (a : A) → B a → Σ A B
````

Compare this with conjunction, which corresponds to non-dependent product $A \wedge B$:

```agda
data _∧_ (A : Set) (B : Set) : Set where
⟨_,_⟩ : A → B → A ∧ B
```

In this sense, in intuitionistic logic existential quantification `Σ` generalises conjunction,
which justifies the name dependent product.
(This can create confusion because `Π` is sometimes called dependent product too,
since also `Π` generalises conjunction.)

(Existential quantification also generalises disjunction,
since the type $B_1 \vee B_2$ is isomorphic to $\Sigma\; A\; B$
with $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, `Σ` is sometimes called dependent sum.
However, we will not follow this terminology here.)

````
thereExists : ∀ {A : Set} (B : A → Set) → Set
thereExists {A} B = Σ A B

∃-syntax = thereExists
infix 0 ∃-syntax
syntax ∃-syntax (λ x → B) = ∃[ x ] B

-- aka uncurry
∃-elim : {A : Set} {B : A → Set} {C : Set} → (∀ (a : A) → B a → C) → Σ A B → C
∃-elim a→b→c ⟨ a , b ⟩ = a→b→c a b
````

We can also define projection functions for dependent pairs.
````
dfst : {A : Set} {B : A → Set} → Σ A B → A
dfst ⟨ a , _ ⟩ = a

dsnd : {A : Set} {B : A → Set} → (p : Σ A B) → B (dfst p)
dsnd ⟨ _ , b ⟩ = b
````

We note how the definition of the functions `dfst` and `dsnd` is exactly as before with non-dependent pairs,
however the *type* of these functions is much more precise now.

### **Exercise** (`∃`)

1. Show that the existential quantifier commutes with itself:
$(\exists a. \exists b. P\; a\; b) \to \exists b. \exists a. P\; a\; b$.


In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell8 where
open import code.cell7 public

∃-comm : {A B : Set} {P : A → B → Set} → ∃[ a ] ∃[ b ] P a b → ∃[ b ] ∃[ a ] P a b
∃-comm = ?


### **Exercise** (`∃` and `→`)

Consider the following two implications (where $a$ is not free in $P$):
1. $(\exists a. P \to Q\; a) \to (P \to \exists a. Q\; a)$, and.
2. $(P \to \exists a. Q\; a) \to (\exists a. P \to Q\; a)$.
They are both valid in classical logic.
Find out which one is valid in intuitionistic logic, and prove it.


In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell9 where
open import code.cell8 public

∃→1 : {A : Set} {P : Set} {Q : A → Set} → ∃[ a ] (P → Q a) → P → ∃[ a ] Q a
∃→1 = ?

∃→2 : {A : Set} {P : Set} {Q : A → Set} → (P → ∃[ a ] Q a) → ∃[ a ] (P → Q a)
∃→2 = ?

### **Exercise** (`∃` and `∨`)


In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell10 where
open import code.cell9 public

-- 1
duncurry : {A : Set} {B : A → Set} {C : Set} → ∀[ a ] (B a → C) → ∃[ a ] B a → C
duncurry = ?

dcurry : {A : Set} {B : A → Set} {C : Set} → (∃[ a ] B a  → C) → ∀[ a ] (B a → C)
dcurry = ?

### **Exercise** (`∃`, `∧`, and `∨`)

Establish whether the following are intuitionistic tautologies
and prove it for the positive cases:

1. $\exists a B \vee C \to (\exists a B) \vee (\exists a C)$.
2. $(\exists a B) \vee (\exists a C) \to \exists a B \vee C$.
3. $\exists a B \wedge C \to (\exists a B) \wedge (\exists a C)$.
4. $(\exists a B) \wedge (\exists a C) \to \exists a B \wedge C$.
5. $(\exists a B) \wedge C \to \exists a B \wedge C$, where $a$ does not occur in $C$.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell11 where
open import code.cell10 public

-- 1
∃∨-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∨ C a → (∃[ a ] B a) ∨ (∃[ a ] C a)
∃∨-distr = ?

-- 2
∨∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∨ (∃[ a ] C a) → ∃[ a ] B a ∨ C a
∨∃-distr = ?

-- 3
∃∧-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∧ C a → (∃[ a ] B a) ∧ (∃[ a ] C a)
∃∧-distr = ?

-- 4
∧∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∧ (∃[ a ] C a) → ∃[ a ] B a ∧ C a
∧∃-distr = ?

-- 5
∧∃-distr' : {A : Set} {B : A → Set} {C : Set} → (∃[ a ] B a) ∧ C → ∃[ a ] B a ∧ C
∧∃-distr' = ?

### **Exercise** (`∀`, `∃`, and `¬`)

Which of the following hold in intuitionistic logic? Prove those that hold.

1. $\exists a \forall b C \to \forall b \exists a C$, where $C$ depends on $a$ and $b$.
2. $\neg \exists a B \to \forall a \neg B$, where $B$ depends on $a$.
3. $\forall a \neg B \to \neg \exists a B$, where $B$ depends on $a$.
4. $\exists a \neg B \to \neg \forall a B$, where $B$ depends on $a$.
5. $\neg \forall a B \to \exists a \neg B$, where $B$ depends on $a$.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell12 where
open import code.cell11 public

-- 1
-- B cannot depend on (a : A) for the swap to be possible!
∃∀-distr : {A : Set} {B : Set} {C : A → B → Set} → ∃[ a ] ∀[ b ] C a b → ∀[ b ] ∃[ a ] C a b
∃∀-distr = ?

-- 2
¬∃→∀¬ : {A : Set} {B : A → Set} → ¬ (∃[ a ] B a) → ∀[ a ] ¬ B a
¬∃→∀¬ = ?

-- 3
∀¬→¬∃ : {A : Set} {B : A → Set} → ∀[ a ] ¬ B a → ¬ (∃[ a ] B a)
∀¬→¬∃ = ?

-- 4
∃¬→¬∀ : {A : Set} {B : A → Set} → ∃[ a ] ¬ B a → ¬ (∀[ a ] B a)
∃¬→¬∀ = ?

-- 5
¬∀→∃¬ : {A : Set} {B : A → Set} → ¬ (∀[ a ] B a) → ∃[ a ] ¬ B a
¬∀→∃¬ = ?

### **Exercise** (`∀`, `∃`, and `→`)

Show that one can always push the existential quantifier inside an implication:
$(\exists a. P\; a \to Q\; a) \to ((\forall a. P\; a) \to \exists a. Q\; a)$.
However, the converse implication does not hold intuitionistically, even when $Q$ does not contain $a$ free:
$((\forall a. P\; a) \to Q) \to \exists a. P\; a \to Q$.


In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell13 where
open import code.cell12 public

∃∀→ : ∀ {A : Set} {P Q : A → Set} → (∃[ a ] (P a → Q a)) → (∀[ a ] P a) → (∃[ a ] (Q a))
∃∀→ = ?

# Binary relations

We apply first-order logic to study some properties of binary relations.
We first define what is a binary relation.

````
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell14 where
open import code.cell13 public

Rel : Set → Set → Set1
Rel A B = A → B → Set
````

(The type of `Rel A B` is `Set1`, which is the type of `Set : Set1`, which is the same as `Set0`.
We do not need to bother anymore with type levels.)

For example, we can express common properties of binary relations.

````
reflexive : ∀ {A : Set} → Rel A A → Set
reflexive R = ∀[ x ] R x x

symmetric : ∀ {A : Set} → Rel A A → Set
symmetric R = ∀[ x ] ∀[ y ] (R x y → R y x)

transitive : ∀ {A : Set} → Rel A A → Set
transitive R = ∀[ x ] ∀[ y ] ∀[ z ] (R x y → R y z → R x z)
````

### **Exercise** (Total relations)

1. Define what it means for a relation $R$ to be total:
$\forall x. \exists y. R x y$.

2. Formalise and prove that every relation $R$ which is symmetric, transitive, and total,
it is also necessarily reflexive.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell15 where
open import code.cell14 public

total : ∀ {A B : Set} → Rel A B → Set
total R = ?

sym∧trans∧tot→refl : (A : Set) (R : Rel A A) → ?
sym∧trans∧tot→refl = ?

## Composition of relations

We can define the composition of binary relations.

````
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell16 where
open import code.cell15 public

_∘_ : ∀ {A B C : Set} → Rel A B → Rel B C → Rel A C
(R ∘ S) a c = ∃[ b ] (R a b ∧ S b c)
````

### **Exercise**
Show that the composition of reflexive relations is also reflexive.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell17 where
open import code.cell16 public

∘-refl : ∀ {A : Set} {R S : Rel A A} → reflexive R → reflexive S → reflexive (R ∘ S)
∘-refl = ?

## Inclusion

We can define that one relation is included into another.

````
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell18 where
open import code.cell17 public

_⊆_ : ∀ {A B : Set} → Rel A B → Rel A B → Set
R ⊆ S = ∀[ x ] ∀[ y ] (R x y → S x y)
````

With this in hand, we can define that one relation commutes with another.

````
commute : ∀ {A : Set} (R S : Rel A A) → Set
commute R S = (R ∘ S) ⊆ (S ∘ R)
````

### **Exercise**
Prove that if $R$ and $S$ are transitive binary relations
and moreover $S$ commutes over $R$, then their composition $R \circ S$ is also transitive.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.cell19 where
open import code.cell18 public

∘-trans : ∀ (A : Set) (R S : Rel A A) → transitive R → transitive S → commute S R → transitive (R ∘ S)
∘-trans = ?